# HealthML-Toolkit: Healthcare Chatbot Module

## Overview

This notebook implements an **NLTK-based conversational AI** for healthcare queries, symptom analysis, and appointment scheduling. It's one of three core modules in the HealthML-Toolkit system.

### What It Does

1. **Pattern-Matching Conversations**: Uses regex-based patterns to understand user queries about symptoms, appointments, and health advice
2. **Intent Detection**: Identifies emergency situations, appointment requests, and general health inquiries
3. **Conversation Persistence**: Automatically saves conversations to binary files with timestamps
4. **PDF Export**: Generate professional appointment summaries and chat history reports

### Key Features

- **Emergency Detection**: Recognizes serious symptoms (chest pain, severe bleeding) and escalates appropriately
- **Personalized Responses**: Uses capture groups and pronoun transformations for natural conversations
- **Appointment Tracking**: Records appointment requests with timestamps for follow-up
- **Extensible Pattern Library**: Easy to add new medical patterns and responses

### Architecture

```
User Input
    ↓
Regex Pattern Matching (pairs list)
    ↓
Intent Classification (symptom/appointment/emergency)
    ↓
Response Generation (with %1, %2 placeholders)
    ↓
Conversation Storage (.bin files)
    ↓
Optional PDF Export
```

### Usage

1. **Run this notebook**: Execute cells sequentially
2. **CLI mode**: `python main.py chatbot` from the project root
3. **Integrate**: Import functions in other Python scripts

### Outputs

- `saved_conversations/appointments_YYYYMMDD_HHMMSS.bin` - Binary conversation logs
- `saved_conversations/appointments_summary.pdf` - PDF reports (optional)

### Related Modules

- **Data Quality**: `data_quality/data_imputation_methods.ipynb` - Imputation for missing patient data
- **Vision**: `vision/diabetic_retinopathy_pipeline.ipynb` - Retinopathy classification

---

**Let's get started! 🏥💬**

In [9]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
from nltk.chat.util import Chat, reflections

In [11]:
pairs = [
    [
        r"(hi|hello|hey)",
        ["Hello! I am your healthcare assistant. How can I help you today?"]
    ],
    [
        r"(.*) (fever|temperature|high temperature)",
        ["It sounds like you might have a fever. Please stay hydrated and consult a doctor if it persists."]
    ],
    [
        r"(.*) (headache|migraine)",
        ["Headaches can have many causes, including stress and dehydration. If severe or recurring, consult a healthcare professional."]
    ],
    [
        r"(.*) (cough|cold)",
        ["A cough could be due to allergies, infection, or irritation. If it lasts more than a week, see a doctor."]
    ],
    [
        r"(.*) chest pain",
        ["Chest pain can be serious. Please seek immediate medical attention."]
    ],
    [
        r"i need an appointment with (.*)",
        ["Sure, I can help you schedule an appointment with %1. Could you provide your preferred date and time?"]
    ],
    [
        r"(.*) healthy diet",
        ["A healthy diet includes fruits, vegetables, whole grains, lean proteins, and sufficient water."]
    ],
    [
        r"(.*) exercise",
        ["Regular exercise (30 minutes a day) improves cardiovascular health and overall well-being."]
    ],
    [
        r"(.*) emergency",
        ["If this is an emergency, please call your local emergency number immediately."]
    ],
    [
        r"quit",
        ["Take care of your health. Goodbye!"]
    ]
]


In [12]:
import nltk
from nltk.chat.util import Chat, reflections
import pickle
from datetime import datetime

appointments = []

def healthcare_chatbot():
    conversation_log = []
    print("Healthcare Assistant: Hello! I am here to provide general health guidance. Type 'quit' to exit.")
    chat = Chat(pairs, reflections)
    
    while True:
        user_input = input("You: ")
        conversation_log.append(("User", user_input))
        
        if user_input.lower() == 'quit':
            break
            
        response = chat.respond(user_input)
        conversation_log.append(("Assistant", response))
        print("Healthcare Assistant:", response)
        
        # Save appointment details if detected
        if "appointment" in user_input.lower():
            appointments.append({
                'datetime': datetime.now().strftime("%Y-%m-%d %H:%M"),
                'request': user_input,
                'response': response
            })
    
    # Save to binary file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"appointments_{timestamp}.bin"
    with open(filename, 'wb') as f:
        pickle.dump({
            'conversation': conversation_log,
            'appointments': appointments
        }, f)
    print(f"\nConversation and appointments saved to {filename}")

if __name__ == "__main__":
    healthcare_chatbot()

Healthcare Assistant: Hello! I am here to provide general health guidance. Type 'quit' to exit.

Conversation and appointments saved to appointments_20250829_153337.bin


In [14]:
def read_appointments(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    
    print("\nSaved Appointments:")
    for apt in data['appointments']:
        print(f"Date/Time: {apt['datetime']}")
        print(f"Request: {apt['request']}")
        print(f"Response: {apt['response']}")
        print("-" * 50)